In [3]:
%pip install numpy
%pip install tqdm
%pip install plotly.express
%pip install pandas

import numpy as np
from tqdm import tqdm
import plotly.express as px
import pandas as pd
from data.preprocessing import preprocess_csv

In [4]:
class GMM(object):

    def __init__(self, X, K, max_iters=100):
        self.points = X
        self.max_iters = max_iters
        self.N = self.points.shape[0]
        self.D = self.points.shape[1]
        self.K = K

    def softmax(self, logit):
        logit = logit - np.max(logit, axis=1, keepdims=True)
        e = np.exp(logit)
        return e / np.sum(e, axis=1, keepdims=True)

    def logsumexp(self, logit):
        maxRows = np.max(logit, axis=1, keepdims=True)
        logit = logit - maxRows
        logit = np.log(np.sum(np.exp(logit), axis=1)).reshape(-1, 1)
        return logit + maxRows

    def normalPDF(self, points, mu_i, sigma_i):
        variances = sigma_i.diagonal()
        """
        points = points - mu_i
        points = np.square(points)
        points = (points * -1/2) / np.square(variances)
        points = np.exp(points)
        points = points * (1 / np.sqrt(2 * np.pi * np.square(variances)))
        print(np.prod(points, axis=1))
        #return np.prod(points, axis=1)
        """
        pdf = []
        #print(points[:,0])
        #print(mu_i[0])
        for i in range(points.shape[1]):
            exponent = points[:,i] - mu_i[i]
            exponent = (np.square(exponent)).reshape(-1, 1)
            exponent = ((exponent * -.5) / variances)[:,i]
            exponent = np.exp(exponent).reshape(-1, 1)
            val = (exponent * (1 / np.sqrt(2 * np.pi * variances)))[:, i]
            pdf.append(val)

        pdf = np.asarray(pdf)
        return np.prod(pdf, axis=0)


    def create_pi(self):
        return np.ones((self.K,)) * 1/self.K

    def create_mu(self):
        indices = np.random.choice(self.points.shape[0], size=self.K, replace=True)
        return self.points[indices]

    def create_sigma(self):
        matrix = []
        for i in range(self.K):
            matrix.append(np.eye(self.D))
        return np.asarray(matrix)

    def _init_components(self, **kwargs):
        np.random.seed(5)
        return self.create_pi(), self.create_mu(), self.create_sigma()

    def _ll_joint(self, pi, mu, sigma, **kwargs):
        arr = []
        for i in range(self.K):
            pdf = self.normalPDF(self.points, mu[i], sigma[i])
            arr.append(np.log(pdf + 1e-32) + np.log(pi[i] + 1e-32))
        return np.array(arr).T

    def _E_step(self, pi, mu, sigma, **kwargs):
        return self.softmax(self._ll_joint(pi, mu, sigma))

    def _M_step(self, tau, **kwargs):
        Nk = np.sum(tau, axis=0)
        pi = Nk / tau.shape[0]

        mu = []
        for i in range(self.K):
            mu.append(np.sum(np.reshape(tau[:,i], (-1, 1)) * self.points, axis=0) / Nk[i])
        mu = np.asarray(mu)

        sigma = []
        for i in range(self.K):
            a = np.reshape(tau[:, i], (tau.shape[0], 1)) * (self.points - np.reshape(mu[i], (1, self.D)))
            b = a.T @ (self.points - np.reshape(mu[i], (1, self.D)))
            c = b / Nk[i]
            d = np.diag(np.diag(c))
            sigma.append(d)
        sigma = np.asarray(sigma)

        return pi, mu, sigma

    def __call__(
        self, abs_tol=1e-16, rel_tol=1e-16, **kwargs
    ):
        pi, mu, sigma = self._init_components(**kwargs)
        pbar = tqdm(range(self.max_iters))

        prev_loss = None
        for it in pbar:
            # E-step
            tau = self._E_step(pi, mu, sigma)
            # M-step
            pi, mu, sigma = self._M_step(tau)
            # calculate the negative log-likelihood of observation
            joint_ll = self._ll_joint(pi, mu, sigma)
            loss = -np.sum(self.logsumexp(joint_ll))
            if it:
                diff = np.abs(prev_loss - loss)
                if diff < abs_tol and diff / prev_loss < rel_tol:
                    break
            prev_loss = loss
            pbar.set_description("iter %d, loss: %.4f" % (it, loss))
        return tau, (pi, mu, sigma)

In [5]:
dataset_original = pd.read_csv('data/data.csv', encoding='latin1')
preprocessed_dataset = preprocess_csv('data/data.csv', ['StockCode', 'InvoiceDate'])
preprocessed_dataset

,InvoiceNo,Description,Quantity,UnitPrice,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6.0,2.55,17850.0,35
1,536365,WHITE METAL LANTERN,6.0,3.39,17850.0,35
2,536365,CREAM CUPID HEARTS COAT HANGER,8.0,2.75,17850.0,35
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,3.39,17850.0,35
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6.0,3.39,17850.0,35
...,...,...,...,...,...,...
258047,560025,PACK OF 60 SPACEBOY CAKE CASES,2.0,0.55,18283.0,35
258048,560025,PACK OF 60 DINOSAUR CAKE CASES,2.0,0.55,18283.0,35
258049,560025,PACK OF 12 WOODLAND TISSUES,2.0,0.29,18283.0,35
258050,560025,PACK OF 12 SUKI TISSUES,1.0,0.29,18283.0,35


In [6]:
dataframe = preprocessed_dataset[["Quantity", "UnitPrice"]]

dataframe

,Quantity,UnitPrice
0,6.0,2.55
1,6.0,3.39
2,8.0,2.75
3,6.0,3.39
4,6.0,3.39
...,...,...
258047,2.0,0.55
258048,2.0,0.55
258049,2.0,0.29
258050,1.0,0.29


In [7]:
dataframe = dataframe.drop_duplicates()

fig = px.scatter(dataframe, x='Quantity', y='UnitPrice')
fig.show()

In [8]:
dataframe

,Quantity,UnitPrice
0,6.0,2.55
1,6.0,3.39
2,8.0,2.75
5,2.0,7.65
6,6.0,4.25
...,...,...
257519,18.0,12.75
257611,144.0,4.15
257618,112.0,10.95
257620,66.0,8.15


In [9]:
dataframe = dataframe[(dataframe < 3000).all(axis=1)]
npdata = dataframe.to_numpy()

dataframe

,Quantity,UnitPrice
0,6.0,2.55
1,6.0,3.39
2,8.0,2.75
5,2.0,7.65
6,6.0,4.25
...,...,...
257519,18.0,12.75
257611,144.0,4.15
257618,112.0,10.95
257620,66.0,8.15


In [10]:
fig = px.scatter(dataframe, x='Quantity', y='UnitPrice')
fig.show()

In [67]:
gmm = GMM(npdata, 5, 10)

tau, var = gmm()

maxes = tau.argmax(axis=1)

maxes


iter 9, loss: 40540.7380: 100%|██████████| 10/10 [00:00<00:00, 90.98it/s]


array([0, 0, 0, ..., 3, 1, 0])

In [68]:
colored_data = np.hstack((npdata, maxes.reshape((-1, 1))))
colored_data = np.hstack((colored_data, np.log10(npdata[:,1].reshape(-1, 1))))

colored_data

<ipython-input-68-155be8d9d167>:2: RuntimeWarning:

divide by zero encountered in log10



array([[  6.        ,   2.55      ,   0.        ,   0.40654018],
       [  6.        ,   3.39      ,   0.        ,   0.5301997 ],
       [  8.        ,   2.75      ,   0.        ,   0.43933269],
       ...,
       [112.        ,  10.95      ,   3.        ,   1.03941412],
       [ 66.        ,   8.15      ,   1.        ,   0.91115761],
       [ 18.        ,   0.21      ,   0.        ,  -0.67778071]])

In [69]:
colored_dataframe = pd.DataFrame(colored_data, columns=["Quantity", "UnitPrice", "Assignment", "LogPrice"])


fig = px.scatter(colored_dataframe, x="Quantity", y="UnitPrice", color="Assignment")
fig.show()

In [70]:
fig = px.scatter(colored_dataframe, x="Quantity", y="LogPrice", color="Assignment")
fig.show()

In [71]:
def calculate_silhouette_score(points, assignments, centers):
    n_points = points.shape[0]
    silhouette_scores = np.zeros(n_points)
    for i in range(n_points):
        current_cluster = assignments[i]
        same_cluster_points = points[assignments == current_cluster]
        if len(same_cluster_points) > 1:
            a_i = np.mean(np.linalg.norm(same_cluster_points - points[i], axis=1))
        else:
            a_i = 0
        b_i = float('inf')
        for k in range(centers.shape[0]):
            if k != current_cluster:
                other_cluster_points = points[assignments == k]
                if other_cluster_points.size > 0:
                    avg_dist_to_other_cluster = np.mean(np.linalg.norm(other_cluster_points - points[i], axis=1))
                    b_i = min(b_i, avg_dist_to_other_cluster)
        if max(a_i, b_i) > 0:
            silhouette_scores[i] = (b_i - a_i) / max(a_i, b_i)
        else:
            silhouette_scores[i] = 0
    return np.mean(silhouette_scores)
silhouette_score = calculate_silhouette_score(npdata, maxes, var[1])

silhouette_score

0.3992979816652339